In [26]:
ecrime_user = ""
ecrime_password = ""
ecrime_key = ""

# Aggregator of crawlers and lists of  ransomware groups (+leaksites)

| # |Public? | Initiative 	 | URL 	|
|---|--------|------------	 | -----	|
| 1 | Y      |NoMoreRansom.org| https://www.nomoreransom.org/ |
| 2 | Y      |Ransom-DB.com   | https://www.ransom-db.com/ransomware-groups |
| 3 | N      |ecrime.ch       | https://ecrime.ch/app/leaksites.php |
| 4 | Y      |Ransom Watch    | https://ransomwatch.telemetry.ltd/#/INDEX	|
| 5 | Y      |Github @fastfire deepdarkCTI| https://github.com/fastfire/deepdarkCTI/blob/main/ransomware_gang.md |
| 6 | Y      |Reddit r/Malware| https://www.reddit.com/r/Malware/comments/qhse7t/list_of_ransomware_groups_and_their_pr_pages_2021/ |
| 7 | Y          	|"Ransomware Group Sites"| http://ransomwr3tsydeii4q43vazm7wofla5ujdajquitomtd47cxjtfgwyyd.onion.pet |
| 8 | Y      |Darkfeed.io    | https://darkfeed.io/	|

Things to do:
- https://malpedia.caad.fkie.fraunhofer.de/
- https://cracked.io/Thread-Other-RANSOMWARE-GROUP-LINKS

NOTE: the domains '.pet' 

Excluded:
- https://ransom.wiki/ == https://twitter.com/ransomwatcher because it is mainly focussed on the victims

### Methodology

How do we **equalize** ransomware group names?
- lower case letters
- remove space between words
- remove '_' between words
- remove '...'
- remove '?'
- '1.0' will become '1', '2.0' will become '2', and '3.0' will become '3'
- in '\<something1>/\<something2>', '\<something2>' will be part of the other_names column
- in '(aka \<something>)' \<something> will be part of other_names column

Types of crawlers:
- using cfscraper and xpath
- using cfscraper and pd.read_table
- using selenium
- using selenium for TOR

In [27]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

## Crawler ransomwr3tsydeii4q43vazm7wofla5ujdajquitomtd47cxjtfgwyyd.onion

In [55]:
import cfscrape
from lxml import etree

def tor_ransomware_group_sites ():
    
    url="http://ransomwr3tsydeii4q43vazm7wofla5ujdajquitomtd47cxjtfgwyyd.onion.pet"
    
    header = {'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9',
              'Accept-Encoding': 'gzip, deflate, sdch',
              'Accept-Language' : 'nl-NL,nl;q=0.8,en-US;q=0.6,en;q=0.4',
              'Cache-Control' : 'max-age=0',
              'Connection': 'keep-alive',
              'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/43.0.2357.81 Safari/537.36'}

    scraper = cfscrape.create_scraper()
    scraped_html=scraper.get(url,headers=header).content

    html = etree.HTML(scraped_html)

    elements = html.xpath("//div[@class='table-body']/div[@class='table-row']")

    # print(scraped_html.decode("utf-8"))

    ransomware_links = pd.DataFrame()

    for element in elements:
        actor = element.xpath("div[@class='table-cell']/text()")[0]
        urls = element.xpath("div[@class='table-cell']/a/@href") 

        for url in urls:
            ransomware_links = pd.concat([ransomware_links , pd.DataFrame.from_records([{'actor':actor,'url':url.replace('//','http://').replace('.pet','')}])])
       
    return ransomware_links

In [56]:
df_tor_info = tor_ransomware_group_sites()
df_tor_info.to_csv('data/ransomware_group_url_list_tor.csv')

## Crawler ecrime.ch: https://ecrime.ch/app/leaksites.php

In [28]:
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
import time

def ecrime_ch (ecrime_user, ecrime_password):
    driver = webdriver.Chrome(ChromeDriverManager().install())

    url = "https://ecrime.ch/app/login.php" 
    driver.get(url)
    time.sleep(2)
    
    username = driver.find_element_by_name("email_address")
    password = driver.find_element_by_name("password")
    username.send_keys(ecrime_user)
    password.send_keys(ecrime_password)
    driver.find_element_by_xpath("//input[@class='btn btn-primary']").click()
    time.sleep(2)
    
    url = "https://ecrime.ch/app/leaksites.php" 
    driver.get(url)
    driver.find_element_by_xpath('//button[text()="⤵ Save as CSV"]').click()
    time.sleep(2)
    
    df_ecrimech = pd.read_csv('~/Downloads/ecrime.ch-leaksites.csv')
    df_ecrimech = df_ecrimech.drop(columns=['Status','Last Seen (UTC)']).rename(columns={'Actor/Malware':'actor', 'URL':'url'})    
    df_ecrimech['actor'] = equalize_names(df_ecrimech['actor'])
    df_ecrimech['other_names'] = df_ecrimech['actor'].dropna().apply(lambda x: x.split('/')[1] if '/'in x else '')
    df_ecrimech['actor'] = df_ecrimech['actor'].dropna().apply(lambda x: x.split('/')[0])

    !rm '/Users/nw/Downloads/ecrime.ch-leaksites.csv'
    
    driver.quit()
    
    return df_ecrimech

In [29]:
df_ecrime = ecrime_ch (ecrime_user, ecrime_password)
df_ecrime.to_csv('data/ransomware_group_url_list_ecrimech.csv')



====== WebDriver manager ======
Current google-chrome version is 106.0.5249
Get LATEST driver version for 106.0.5249
Driver [/Users/nw/.wdm/drivers/chromedriver/mac64/106.0.5249.61/chromedriver] found in cache


In [44]:
df_ecrime

,Unnamed: 0,actor,url,other_names,all_actor_names
1,1,54bb47h,http://54bb47h5qu4k7l4d7v5ix3i6ak6elysn3net4by4ihmvrhu7cvbskoqd.onion/,sabbath,[54bb47h]
2,2,54bb47h,http://54bb47h.blog/,sabbath,[54bb47h]
47,47,d0n#t::leaks::,https://sbc2zv2qnz5vubwtx3aobfpkeao6l4igjegm3xx7tk5suqhjkp5jxtqd.onion/,donutleaks,[d0n#t::leaks::]


## Crawler NoMoreRansom Ransomware Groups (that have decription tools)

In [56]:
import pandas as pd
import cfscrape
from lxml import etree

def nomoreransom ():
    url = "https://www.nomoreransom.org/en/decryption-tools.html" 
    
    header = {'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9',
          'Accept-Encoding': 'gzip, deflate, sdch',
          'Accept-Language' : 'nl-NL,nl;q=0.8,en-US;q=0.6,en;q=0.4',
          'Cache-Control' : 'max-age=0',
          'Connection': 'keep-alive',
          'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/43.0.2357.81 Safari/537.36'}

    scraper = cfscrape.create_scraper()
    scraped_html=scraper.get(url,headers=header).content
    html = etree.HTML(scraped_html)

    
    df = pd.DataFrame()
    elements = html.xpath("//button[@class='expandable-toggler']")
    for element in elements:
        df = pd.concat([df , pd.DataFrame.from_records([{'actor':element.text}])])
    
    return df

In [57]:
df_nomoreransom = nomoreransom ()
df_nomoreransom.to_csv('data/ransomware_group_list_nomoreransom.csv')

## Get STATIC https://www.reddit.com/r/Malware/comments/qhse7t/list_of_ransomware_groups_and_their_pr_pages_2021/

In [4]:
url = "https://www.reddit.com/r/Malware/comments/qhse7t/list_of_ransomware_groups_and_their_pr_pages_2021/" 

import cfscrape
from lxml import etree

scraper = cfscrape.create_scraper()
scraped_html=scraper.get(url).content
html = etree.HTML(scraped_html)
elements = html.xpath("//div[@class='_3xX726aBn29LDbsDtzr_6E _1Ap4F5maDtT1E1YuCiaO0r D3IL3FD0RFy_mkKLPwL4']/div/p")

df_reddit = pd.DataFrame()
for element in elements:
    try:
        actor = element.text.replace(':','')
        url = element.xpath("a/@href")[0]
        df_reddit = pd.concat([df_reddit , pd.DataFrame.from_records([{'actor':actor, 'url':url}])])
    except:
        next

In [6]:
df_reddit.to_csv('data/ransomware_group_url_list_reddit2021.csv')

## Crawl Ransom DB https://www.ransom-db.com/ransomware-groups

In [65]:
url = "https://www.ransom-db.com/ransomware-groups" 

import cfscrape
from lxml import etree

scraper = cfscrape.create_scraper()
scraped_html=scraper.get(url).content
tables = pd.read_html(scraped_html)

df_ransomdb = tables[0].drop(columns=['No.','Description','Last Incident \ Victim','Victim Count','Last Seen','Icon']).rename(columns={'Group Name':'actor'})    
df_ransomdb.to_csv('data/ransomware_group_list_ransomdb.csv')

## Crawler Ransom Watch: https://ransomwatch.telemetry.ltd/#/INDEX
It is a Jekyll based on https://raw.githubusercontent.com/joshhighet/ransomwatch/5d5caf0542d9ca5a225e15fa1cbf8b404a013d31/docs/INDEX.md

NOTE: the @joshhighet contacted us to change the crawler for a better solution. 
```
in lieu of parsing markdown you may find it easier to just leverage the files that generate them, http://ransomwhat.telemetry.ltd/posts & http://ransomwhat.telemetry.ltd/groups
```
THANKS! We've decided to comment the crawler.

NOTE: http://ransomwhat.telemetry.ltd/groups points to https://raw.githubusercontent.com/joshhighet/ransomwatch/main/groups.json


In [201]:
# url = 'https://raw.githubusercontent.com/joshhighet/ransomwatch/5d5caf0542d9ca5a225e15fa1cbf8b404a013d31/docs/INDEX.md'

# df_ransomwatch = pd.read_table(url, sep="|", header=1, skipinitialspace=True).dropna(axis=1, how='all').iloc[1:]
# df_ransomwatch = df_ransomwatch.drop(columns=['title ','status ','last seen ']).rename(columns={'group ':'actor','location ':'url'})
# df_ransomwatch['actor'] = df_ransomwatch['actor'].apply(lambda x: x.split(']')[0].replace('[',''))

In [35]:
df_ransomwatch = pd.read_json("https://raw.githubusercontent.com/joshhighet/ransomwatch/main/groups.json")

df_ransomwatch['url'] = [[d.get('fqdn') for d in x] for x in df_ransomwatch['locations']]

df_ransomwatch = df_ransomwatch.drop(columns=['captcha','parser','javascript_render','meta','profile', 'locations']).rename(columns={'name':'actor'})

In [38]:
df_ransomwatch.to_csv('data/ransomware_group_url_list_ransomwatch.csv')

## Crawler Github @fastfire ransomware-gang: https://github.com/fastfire/deepdarkCTI/blob/main/ransomware_gang.md
This is the same as: https://raw.githubusercontent.com/fastfire/deepdarkCTI/main/ransomware_gang.md

ATTENTION: Not all URLs are from ransomware groups!

In [121]:
url = 'https://raw.githubusercontent.com/fastfire/deepdarkCTI/main/ransomware_gang.md'

df_deepdarkcti = pd.read_table(url, sep="|", header=1, skipinitialspace=True).dropna(axis=1, how='all')

df_deepdarkcti = df_deepdarkcti.drop(columns=['------ .1','------ .2','------ .3','------ .4'])

df_deepdarkcti['actor'] = df_deepdarkcti['------ '].apply(lambda x: x.split(']')[0].replace('[',''))
df_deepdarkcti['url'] = df_deepdarkcti['------ '].apply(lambda x: x.split('(')[-1].replace(')',''))

df_deepdarkcti = df_deepdarkcti.drop(columns=['------ '])

toremove = ['RANSOM DB','RANSOMWARE GROUP SITES (list)' ]
df_deepdarkcti = df_deepdarkcti[~df_deepdarkcti['actor'].isin(toremove)]

df_deepdarkcti.to_csv('data/ransomware_group_url_list_deepdarkcti.csv')

## Crawler Darkfeed.io

In [4]:
url = "https://darkfeed.io/ransomgroups/" 

import cfscrape
from lxml import etree

scraper = cfscrape.create_scraper()
scraped_html=scraper.get(url).content
html = etree.HTML(scraped_html)

elements = html.xpath("//div[@class='elementor-widget-container']/h5")

df_darkfeed = pd.DataFrame()
for element in elements:
    try:
        actor = element.xpath("a/text()")[0]
        if actor == 'Telegram':
            next
        url = element.xpath("a/@href")[0]
        df_darkfeed = pd.concat([df_darkfeed , pd.DataFrame.from_records([{'actor':actor, 'url':url}])])
    except:
        next

df_darkfeed.reset_index(drop=True, inplace=True)
df_darkfeed.to_csv('data/ransomware_group_url_list_darkfeed.csv')

## ====================

In [39]:
import pandas as pd

In [56]:
def equalize_names (df_series):   
    return df_series.str.lower()\
                    .str.replace("former ",'',regex=False)\
                    .str.replace("aka ",'',regex=False)\
                    .str.replace("(victims page)","",regex=False)\
                    .str.replace("(victims request id page)","",regex=False)\
                    .str.replace("(support page)","",regex=False)\
                    .str.replace("(tor v3)","",regex=False)\
                    .str.replace("(tor v2)","",regex=False)\
                    .str.replace("(1)","",regex=False)\
                    .str.replace("(2)","",regex=False)\
                    .str.replace("(3)","",regex=False)\
                    .str.replace("(4)","",regex=False)\
                    .str.replace("(old)","",regex=False)\
                    .str.replace("(new)","",regex=False)\
                    .str.replace("(deep)","",regex=False)\
                    .str.replace("support","",regex=False)\
                    .str.replace("(dark)","",regex=False)\
                    .str.replace("loker","",regex=False)\
                    .str.replace(" ","",regex=False)\
                    .str.replace("_","",regex=False)\
                    .str.replace("?","",regex=False)\
                    .str.replace('\.\.\.',"",regex=False)\
                    .str.replace('"','',regex=False)\
                    .str.replace("'","",regex=False)\
                    .str.replace("group","",regex=False)\
                    .str.replace("ransomware","",regex=False)\
                    .str.replace("ransom","",regex=False)\
                    .str.replace("2.0","2",regex=False)\
                    .str.replace("3.0","3",regex=False)

def split_all_name (df_series):
    actors=[]
    for actor in df_series:
        if '/' in actor:
            actors.append(actor.split('/'))
        elif '(' in actor:
            actors.append(actor.replace(')','').split('('))
        else:
            actors.append([actor])
    return actors

### ecrime

In [60]:
df_ecrime = pd.read_csv('data/ransomware_group_url_list_ecrimech.csv')

df_ecrime['all_actor_names'] = split_all_name(\
                                    equalize_names (df_ecrime['actor']))

ValueError: Length of values (174) does not match length of index (175)

### nomoreransom

In [32]:
df_nomoreransom = pd.read_csv('data/ransomware_group_list_nomoreransom.csv')

df_nomoreransom['all_actor_names'] = split_all_name(\
                                    equalize_names (df_nomoreransom['actor']))

### tor node

In [33]:
df_tornode = pd.read_csv('data/ransomware_group_url_list_tor.csv')

df_tornode['all_actor_names'] = split_all_name(\
                                    equalize_names (df_tornode['actor']))

### reddit2021

In [34]:
df_reddit2021 = pd.read_csv('data/ransomware_group_url_list_reddit2021.csv')

df_reddit2021['all_actor_names'] = split_all_name(\
                                    equalize_names (df_reddit2021['actor']))

### deepdarkcti

In [35]:
df_deepdarkcti = pd.read_csv('data/ransomware_group_url_list_deepdarkcti.csv')
df_deepdarkcti['all_actor_names'] = split_all_name(\
                                    equalize_names (df_deepdarkcti['actor']))

### ransomwatch

In [36]:
df_ransomwatch = pd.read_csv('data/ransomware_group_url_list_ransomwatch.csv')
df_ransomwatch['all_actor_names'] = split_all_name(\
                                    equalize_names (df_ransomwatch['actor']))

### ransomdb

In [37]:
df_ransomdb = pd.read_csv('data/ransomware_group_list_ransomdb.csv')
df_ransomdb['all_actor_names'] = split_all_name(\
                                    equalize_names (df_ransomdb['actor']))

### darkfeed.io

In [38]:
df_darkfeed = pd.read_csv('data/ransomware_group_url_list_darkfeed.csv')
df_darkfeed['all_actor_names'] = split_all_name(\
                                    equalize_names (df_darkfeed['actor']))

# COMPARING

In [39]:
ecrime = set([x for l in df_ecrime['all_actor_names'] for x in l])
tornode = set([x for l in df_tornode['all_actor_names'] for x in l])
reddit2021 = set([x for l in df_reddit2021['all_actor_names'] for x in l])
deepdarkcti = set([x for l in df_deepdarkcti['all_actor_names'] for x in l])
ransomwatch = set([x for l in df_ransomwatch['all_actor_names'] for x in l])
nomoreransom = set([x for l in df_nomoreransom['all_actor_names'] for x in l])
ransomdb = set([x for l in df_ransomdb['all_actor_names'] for x in l])
darkfeed = set([x for l in df_darkfeed['all_actor_names'] for x in l])

#### Stats unique names

In [40]:
print("ecrime:",len(ecrime))
print("tornode:",len(tornode))
print("reddit2021:",len(reddit2021))
print("deepdarkcti:",len(deepdarkcti))
print("ransomwatch:",len(ransomwatch))
print("nomoreransom:",len(nomoreransom))
print("ransomdb:",len(ransomdb))
print("darkfeed:",len(darkfeed))

ecrime: 2
tornode: 77
reddit2021: 25
deepdarkcti: 129
ransomwatch: 125
nomoreransom: 127
ransomdb: 54
darkfeed: 49


#### Intersection Matrix

In [62]:
list_initiatives = [
#     'ecrime',
                    'tornode','reddit2021','deepdarkcti','ransomwatch','nomoreransom','ransomdb','darkfeed']

In [63]:
matrix_intersection = pd.DataFrame(columns=list_initiatives,index=list_initiatives)

for initiative1 in list_initiatives:
    for initiative2 in list_initiatives:
        matrix_intersection.loc[initiative1,initiative2] = len(eval(initiative1) & eval(initiative2))

In [64]:
matrix_intersection.apply(pd.to_numeric).style.background_gradient()

,tornode,reddit2021,deepdarkcti,ransomwatch,nomoreransom,ransomdb,darkfeed
tornode,77,23,58,57,6,38,24
reddit2021,23,25,22,20,4,14,10
deepdarkcti,58,22,129,85,12,38,24
ransomwatch,57,20,85,125,8,36,33
nomoreransom,6,4,12,8,127,8,1
ransomdb,38,14,38,36,8,54,18
darkfeed,24,10,24,33,1,18,49
